In [ ]:
#Importing Libraries

import pandas as pd

import string
string.punctuation

#tokenization 

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

#normalization

import time
import logging
import sys

from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

logger = logging.getLogger (__name__)

morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer (morphology)
extractor = TurkishSentenceExtractor()

#lemmatization

import zeyrek

#compute ngrams
from gensim.models import Phrases

#generate a wordcloud image

import numpy as np
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

#importing libraries for lda

import gensim
import pyLDAvis.gensim_models
import gensim.corpora as corpora
from gensim.corpora.dictionary import Dictionary
from gensim.models import CoherenceModel
import numpy as np
from pprint import pprint

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

#visualize the LDA model
import matplotlib.pyplot as plt 
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
#Loading the data

data = pd.read_csv("mydata.csv", error_bad_lines=False, engine ='python')
data.head()

In [2]:
len (data)

42890

In [ ]:
data['user_description'].value_counts()

In [ ]:
data.info()

In [ ]:
#remove punctuation

def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree
#storing the puntuation free text
data['clean']= data['text'].apply(lambda x:remove_punctuation(x))
data.head()

In [ ]:
#lowercase the text

data['lower']= data['clean'].apply(lambda x: x.lower())
data.head()

In [ ]:
#tokenizing text

nltk.word_tokenize('lower', language='turkish')
data['textokenized'] = data.apply(lambda row: nltk.word_tokenize(row['lower']), axis=1)
data.head()

In [ ]:
#removing stopwords

nltk.download ('stopwords')
#Stop words present in the library
stops = nltk.corpus.stopwords.words('turkish')
newstops = ['sa','as', 'ben', 'bende', 'imühamkalp', '2', 'hayır', 'mod', 'cisillll', 'pquraen', 'al', 'mis', 'bent', 'imuhamopustuk', 'senin', 'sen', 'onu', 'bence', 'darkholmee', 'x', 'mu', 'ay', 'naber', 'imuhamiys', 'olur', 'selam', 'insta', 'httpswwwinstagramcomimuhammetyildizhltr', 'haydaaaa', 'hellllooo', 'hellooo', 'hellooooo', 'heee', 'şey', 'haydaaaaaaa', 'haydaaaaaaaa', 'am', 'da', 'daha', 'iyi', 'napalion7', 'bi', 'bosver', 'imuhamalo', 'napalion', 'merhaba', 'httpswwwyoutubecomchannelucstjlfccjuguahupwssiww', 'tutodvrm', 'lul', 'on',  'slm', 'aa', 'şş', 'aaa', 'aaaa', 'aaaaaa', 'aaaaaaa', 'mamiş', 'D', 'çok', 'az', 'neyse', 'falcon2kkalp', 'httpsdiscordggrdj7hxmvgy', ' falcon2kkalp', 'the', 'idil', 'bu', 'httpsdiscordggrdj7hxmvgy', 'neyse', 'ya', 'ne', 'nympea', 'at', 'ays', 'burcinvr', 'san', 'sela', 'değil', 'tutodurm', 'ah', 'be', 'kerema5keremkalp','sö', 'falan', 'bc', 'dc', 'he', 'up', 'aq', 'fizy', '10', 'su', 'nim', 'efe', 'hee', 'hb', 'a', 'gsyi', 'ad', 'ha', 'ner', 'berra', 'hg', 'cafercan3125', 'ayn', 'amk', 'do', 'fa', 'alt','var', 'ba', 'ronaldo', 'ara', 'ada', 'mami', 'bura', 'on', 'fa', 'bb', 'ge', 'ab', 'btw', 'mk', 'pek','Kerem','Keremakdemirr', 'keremakdemirr', 'kere', 'ays055', 'burçin', 'imühamiys', 'mi', 'aç', 'httpswwwyoutubecomchanneluçstjlfçcjuguahupwssiww', 'nympeadansdans','kerem', 'httpsyoutubevsr46l41bru', 'httpswwwinstagramcomkeremakdemirr', 'cafer', 'cüneyt','e', 'i', '5', 'lan', 'la', 'se', 'd', '3', '1', 'bir', 'cafer', 'yay', 'imuhammetyildiz', 'muhamed','deniz', 'Deniz', 'nympea', 'imuhaiys', 'alo', 'evet', 'hayir', 'yok', 'oha']
stops.extend(newstops)

data['nostopwords'] = data['textokenized'].apply(
    lambda x: [word for word in x if word not in stops])
data.head()

In [10]:
#  normalization

def normalize_long_text(text):
    normalized_sentences = [normalizer.normalize(word) for word in text]
    normalized_text = " ".join(normalized_sentences)
    return normalized_text

sentences = data['nostopwords'].copy()
new_sent = []
start = time.time()

for token in sentences:   
    if token.count('') > 0:
        token = list(filter(('').__ne__, token))
    new_token = normalize_long_text(token)
    new_sent.append(new_token)

logger.info(f"Sentences normalized in: {time.time() - start} s")

splitted_words = []
for sent in new_sent:
    words = sent.split()
    splitted_words.append(words)
    
for token in splitted_words:
    j = 0
    for word in token:
        new_word = word.replace('"', '').replace("’", '').replace("'", '').replace("”", '')
        token[j] = new_word
        j += 1


2022-06-03 22:38:22,599 - __main__ - INFO
Msg: Sentences normalized in: 1102.4149286746979 s



In [ ]:
# lemmatization

import warnings
warnings.simplefilter('ignore')
analyzer = zeyrek.MorphAnalyzer()
lem_sent = []
for sent in splitted_words:
    normalized_sent = []
    for word in sent:
        if word == '':
            continue
        else:
            lem_word = analyzer.lemmatize(word)
            normalized_sent.append(lem_word[0][1][0])
    lem_sent.append(normalized_sent)
    
x = lem_sent.copy()
for sent in x:
    i = 0
    for token in sent:
        sent[i] = token.lower()
        i += 1
lem_sent = x


lem_sent = list(filter(('').__ne__, lem_sent))
data['lemmatized'] = lem_sent
data['lemma_str'] = [' '.join(map(str, l)) for l in data['lemmatized']]

data.head(20)

In [ ]:
# Compute bigrams.
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).

bigram = Phrases(data.lemmatized, min_count=20)
for idx in range(len(data.lemmatized)):
    for token in bigram[data.lemmatized[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            data.lemmatized[idx].append(token)

In [ ]:
#remove stopwords again after lemmatization

data['final_text'] = data['lemmatized'].apply(
    lambda x: [word for word in x if word not in stops])
data.head()

In [27]:
#exporting the dataframe to csv
data.to_csv('preprocessed_data.csv')

In [ ]:
# Generate a word cloud image

wordcloud = WordCloud(stopwords=stops, background_color="white", width=800, height=400).generate(text)

# Display the generated image:
# the matplotlib way:

plt.axis("off")
plt.figure( figsize=(40,20))
plt.tight_layout(pad=0)
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

In [ ]:
#LDA 

# Create Dictionary 
id2word = corpora.Dictionary(data['final_text'])  
# Create Corpus 
texts = data['final_text']  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

In [ ]:
#LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the keyword of topics- LDA
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence
from gensim.models import CoherenceModel
# Compute Coherence Score

if __name__ == "__main__":

     coherence_model_lda = CoherenceModel(model=lda_model, texts= data ['final_text'], 
                                                          dictionary=id2word, 
                                                              coherence='u_mass')
     coherence_lda = coherence_model_lda.get_coherence()
     print('\nCoherence Score: ', coherence_lda)

In [24]:
#visualize the LDA model

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.114323  0.002982       1        1  35.142904
1     -0.034316 -0.080952       2        1  23.641069
0     -0.033838  0.038997       3        1  22.390296
3     -0.046169  0.038973       4        1  18.825730, topic_info=            Term        Freq       Total Category  logprob  loglift
37           abi  637.000000  637.000000  Default  30.0000  30.0000
273          cok  668.000000  668.000000  Default  29.0000  29.0000
73         kanka  375.000000  375.000000  Default  28.0000  28.0000
586         sana  306.000000  306.000000  Default  27.0000  27.0000
494        benim  257.000000  257.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
85      selamlar   43.993967   44.742219   Topic4  -6.1838   1.6531
8901        caps   47.868195   48.697851   Topic4  -6.0994   1.6528
4461          to   72.834321   78.467394   Topic4  -5.6796   1.5954
4712      hediye   47.916430   49.907093   Topic4  -6.0984   1.6292
353   imuhamkalp   55.842948  529.043704   Topic4  -5.9453  -0.5786

[171 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
431       2  0.990298     20
9747      1  0.017122     35
9747      2  0.958823     35
9747      3  0.017122     35
9747      4  0.017122     35
...     ...       ...    ...
338       1  0.996210  zaten
1060      1  0.997252    zor
294       1  0.989360   önce
307       1  0.996754   öyle
1010      2  0.985047     öz

[177 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4])

In [ ]:
#What is the dominant topic and its percentage contribution in each document

def format_topics_sentences(lda_model=None, corpus=corpus, texts=data['final_text']):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(lda_model[corpus]):
        row = row_list[0] if lda_model.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = lda_model.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(data['lower'])
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(lda_model=lda_model, corpus=corpus, texts=data['lower'])

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head()

In [ ]:
# 1. Wordcloud of Top N words in each topic

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stops,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=50,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = lda_model.show_topics(formatted=False)

fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
#Plot word count and weights of topic keywords

from collections import Counter
topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data['final_text'] for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        

# Plot Word Count and Weights of Topic Keywords
fig, axes = plt.subplots(2, 2, figsize=(16,10), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
    ax.bar(x='word', height="word_count", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.5, alpha=0.3, label='Word Count')
    ax_twin = ax.twinx()
    ax_twin.bar(x='word', height="importance", data=df.loc[df.topic_id==i, :], color=cols[i], width=0.2, label='Weights')
    ax.set_ylabel('Word Count', color=cols[i])
    ax_twin.set_ylim(0, 0.030); ax.set_ylim(0, 3500)
    ax.set_title('Topic: ' + str(i), color=cols[i], fontsize=16)
    ax.tick_params(axis='y', left=False)
    ax.set_xticklabels(df.loc[df.topic_id==i, 'word'], rotation=30, horizontalalignment= 'right')
    ax.legend(loc='upper left'); ax_twin.legend(loc='upper right')

fig.tight_layout(w_pad=2)    
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=22, y=1.05)    
plt.show()